# Ninja Star

In [ ]:
import pandas as pd

df = pd.read_csv("data/coordinates.csv")

df

## Determine Center

In [ ]:
x_mean, y_mean = df["x"].mean(), df["y"].mean()

In [ ]:
x_mean, y_mean

## Average Identical Points

In [ ]:
df["x_center"] = df["x"] - x_mean
df["y_center"] = df["y"] - y_mean

In [ ]:
df

In [ ]:
from itertools import groupby

x_col, y_col = [], []
for i, row in df.iterrows():
    x, y = row["x_center"], row["y_center"]
    while x < 0.0 or y < 0.0:
        x, y = -y, x
    x_col.append(x)
    y_col.append(y)

In [ ]:
df["x_center_1st"] = x_col
df["y_center_1st"] = y_col

In [ ]:
df

In [ ]:
import numpy as np

df_avg = df.groupby("group_id").agg({"x_center_1st": np.mean, "y_center_1st": np.mean})

In [ ]:
from itertools import groupby
import numpy as np

x_col, y_col = [], []
for i, row in df.iterrows():
    x, y = df_avg.loc[row["group_id"]]
    while np.sign(row["x_center"]) != np.sign(x) or np.sign(row["y_center"]) != np.sign(
        y
    ):
        x, y = -y, x
    x_col.append(x)
    y_col.append(y)

In [ ]:
df["x_avg_center"] = x_col
df["y_avg_center"] = y_col

In [ ]:
df["x_avg"] = df["x_avg_center"] + x_mean
df["y_avg"] = df["y_avg_center"] + y_mean

In [ ]:
df

## Align Points

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from typing import Iterable, Tuple


def regression(df, indexes: Iterable[int]) -> Tuple[float, float]:
    x, y = [], []
    for i in indexes:
        x.append(
            [
                df.loc[i]["x_avg"],
            ]
        )
        y.append(df.loc[i]["y_avg"])

    reg = LinearRegression().fit(x, y)
    return reg.coef_[0], reg.intercept_


def intersection(m1: float, c1: float, m2: float, c2: float) -> Tuple[float, float]:
    x = (c2 - c1) / (m1 - m2)
    y = m1 * x + c1
    return x, y


def get_intersection(
    df, index_1: Iterable[int], index_2: Iterable[int]
) -> Tuple[float, float]:
    m1, c1 = regression(df, index_1)
    m2, c2 = regression(df, index_2)

    return intersection(m1, c1, m2, c2)

In [ ]:
lines = (
    ((14, 15), (14, 13, 45, 46)),
    ((48, 38, 36, 52, 24, 27, 63, 67), (14, 13, 45, 46)),
    ((24, 25), (48, 38, 36, 52, 24, 27, 63, 67)),
    ((24, 25), (25, 26)),
    # ((25,26), (16,19))  # overshoot
    ((63, 64), (16, 19)),
    ((16, 19), (0, 1, 20, 23)),
    ((0, 1, 20, 23), (49, 50)),
    ((49, 50), (51, 50)),
    ((51, 50), (51, 52)),
    ((51, 52), (48, 38, 36, 52, 24, 27, 63, 67)),
    ((36, 37), (48, 38, 36, 52, 24, 27, 63, 67)),
    ((36, 37), (0, 1, 20, 23)),
    ((0, 15), (0, 1, 20, 23)),
    ((0, 15), (14, 15)),
)

In [ ]:
final_points = []

for index_1, index_2 in lines:
    x, y = get_intersection(df, index_1, index_2)
    final_points.append((x, y))

In [ ]:
final_points

In [ ]:
final_points_centered = []

for x, y in final_points:
    x -= x_mean
    y -= y_mean
    final_points_centered.append((x, y))

## Latex Output

In [ ]:
BASENAME = "ninja_star"

In [ ]:
!rm -f {BASENAME + ".pdf"} {BASENAME + ".aux"} {BASENAME + ".log"} {BASENAME + ".tex"}

In [ ]:
points = [f"({x}, {y})" for x, y in final_points_centered]
points.append("cycle")
m_shape = "\\filldraw[black] " + " -- ".join(points) + ";"

In [ ]:
TEMPLATE_BEGIN = r"""\documentclass{article}

\usepackage[utf8]{inputenc}
\usepackage[a4paper, left=2cm, right=2cm, landscape]{geometry}
\usepackage{tikz}

\pagestyle{empty}

\begin{document}

\begin{tikzpicture}

\tikzset{
  m_shape/.pic={
"""

BEGIN_END = """
  }
}

\pic at (0,0) {m_shape};
\pic at (0,0) [rotate=90] {m_shape};
\pic at (0,0) [rotate=180] {m_shape};
\pic at (0,0) [rotate=270] {m_shape};
\end{tikzpicture}

\end{document}
"""

In [ ]:
latex_content = TEMPLATE_BEGIN + m_shape + BEGIN_END

In [ ]:
with open(BASENAME + ".tex", "w") as output:
    output.write(latex_content)

In [ ]:
!pdflatex {BASENAME + ".tex"}